# AllTissueAllSplicingJunctionAnalysis as a Notebook 

rMATS 3.2.5 was run on controlled access RNASeq files retrieved experiments stored in the Sequence Read Archive with controlled access managed by dbGaP.   The data were generated under the Gene Tissue Expression.

## rMATS RNASeq-MATS.py produces 10 different output types which get assembled into as type junction ID by sample ID matrices

### Alternative Splice Site Types are: (se, a3ss, a5ss, mxe, ri)

 This is input as ARGV1 into variable 'astype'

  * Skipped Exon events (se),
  * Alternative 3' splice site (a3ss),
  * Alternative 5' splice site (a5ss),
  * Mutually exclusive exon (mxe),
  * and retention intron (ri)

### There are two different kinds of junction counts

  * jc = junction counts - reads that cross the junction
  * jcec = junction counts plus reads on the target (such as included exon

### And the count type -- there are 5 types

  * inclusion levels (percent spliced in)
  * included junction counts (ijc)
  * skipped junction counts (sjc)
  * inclusion length (inclen)
  * skipped length (skiplen)

### function: fit_iso_tissue 

fit_iso_tissue expects the following input:

  * the tissue of interest (SMSTD) 
  * an ordered_merged_rmats -- which will be ordered to fit the count matrix
  * count matrix (inc or ijc & sjc merged)
  * splice type (a3ss, a5ss, mxe, ri or se)
  * junction_count type (jc or jcec)
  * count type (inc or the merged ijc,sjc)
  
### reordering to match annotations between count matrix and annotation matrix

Common problem is to match specifically the rows of an annotation matrix with the columns of a count matrix
`match` is the function that gives the re-ordering index required to accomplish this


## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## rMATS-final-merged
the rmats-nf NextFlow was executed and the results released here:

## Loading dependencies

In [1]:
library(limma)
library(piggyback)
library(multtest)
library(Biobase)
library(edgeR)
library(tibble)
#install.packages('R.utils')
library(R.utils)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bio

## Analysis 

This analysis uses edgeR.  Normalization takes the form of correction factors computed internally by edgeR functions, but it is also possible for a user to supply them. The correction factors may take the form of scaling factors for the library sizes, such as computed by calcNormFactors, which are then used to compute the effective library sizes. 

In this analysis, we are using the raw counts as provided by rMATS 3.2.5.  The raw counts we are using in the model are `ijc` and `sjc`, the sample specific raw read counts as they align to the junctions of the `included exon (ijc)` and the junctions of the `excluded or skipped exon (sjc)` respectively.


Be sure to set your GITHUB_TOKEN, prior to downloading files

One suggestion is change it to your token and then run it then immediately change it back to this:

Sys.setenv(GITHUB_TOKEN = "your-very-own-github-token")

### Did you remember?
Did you remember to delete your private github token?  Now is a good time to do so, before you save your work and checkit in inadvertantly....

In [2]:
if (!("rmats_final.se.jc.ijc.txt.gz" %in% list.files("../rmats_counts/"))) {    
    # SE
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.inc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.inclen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.skiplen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    # RI
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.ri.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.ri.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.ri.jc.inc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.ri.jc.inclen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.ri.jc.skiplen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    # MXE
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.mxe.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.mxe.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.mxe.jc.inc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.mxe.jc.inclen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.mxe.jc.skiplen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    # A3SS
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a3ss.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a3ss.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a3ss.jc.inc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a3ss.jc.inclen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a3ss.jc.skiplen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
     # A5SS
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a5ss.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a5ss.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a5ss.jc.inc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a5ss.jc.inclen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.a5ss.jc.skiplen.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../rmats_counts/")
} 

## Read in the count data

In [3]:
message("loading a3ss.jc.ijc rMATS 3.2.5 counts\n")
a3ss.jc.ijc <- data.table::fread("../rmats_counts/rmats_final.a3ss.jc.ijc.txt.gz")
message("done!\n")
message("loading a3ss.jc.sjc rMATS 3.2.5 counts\n")
a3ss.jc.sjc <- data.table::fread("../rmats_counts/rmats_final.a3ss.jc.sjc.txt.gz")    
message("done!\n")

message("loading a5ss.jc.ijc rMATS 3.2.5 counts\n")
a5ss.jc.ijc <- data.table::fread("../rmats_counts/rmats_final.a5ss.jc.ijc.txt.gz")    
message("done!\n")
message("loading a5ss.jc.sjc rMATS 3.2.5 counts\n")
a5ss.jc.sjc <- data.table::fread("../rmats_counts/rmats_final.a5ss.jc.sjc.txt.gz")    
message("done!\n")

message("loading mxe.jc.ijc rMATS 3.2.5 counts\n")
mxe.jc.ijc  <- data.table::fread("../rmats_counts/rmats_final.mxe.jc.ijc.txt.gz")    
message("done!\n")
message("loading mxe.jc.sjc rMATS 3.2.5 counts\n")
mxe.jc.sjc  <- data.table::fread("../rmats_counts/rmats_final.mxe.jc.sjc.txt.gz")    
message("done!\n")

message("loading ri.jc.ijc rMATS 3.2.5 counts\n")
ri.jc.ijc   <- data.table::fread("../rmats_counts/rmats_final.ri.jc.ijc.txt.gz")    
message("done!\n")
message("loading ri.jc.sjc rMATS 3.2.5 counts\n")
ri.jc.sjc   <- data.table::fread("../rmats_counts/rmats_final.ri.jc.sjc.txt.gz")    
message("done!\n")

message("loading se.jc.ijc rMATS 3.2.5 counts\n")
se.jc.ijc   <- data.table::fread("../rmats_counts/rmats_final.se.jc.ijc.txt.gz")    
message("done!\n")
message("loading se.jc.sjc rMATS 3.2.5 counts\n")
se.jc.sjc   <- data.table::fread("../rmats_counts/rmats_final.se.jc.sjc.txt.gz")
message("done!\n")

a3ss.jc.ijc[1:5,1:5]
a3ss.jc.sjc[1:5,1:5]

a5ss.jc.ijc[1:5,1:5]
a5ss.jc.sjc[1:5,1:5]

mxe.jc.ijc[1:5,1:5]
mxe.jc.sjc[1:5,1:5]

ri.jc.ijc[1:5,1:5]
ri.jc.sjc[1:5,1:5]

se.jc.ijc[1:5,1:5]
se.jc.sjc[1:5,1:5]

loading a3ss.jc.ijc rMATS 3.2.5 counts

done!

loading a3ss.jc.sjc rMATS 3.2.5 counts

done!

loading a5ss.jc.ijc rMATS 3.2.5 counts

done!

loading a5ss.jc.sjc rMATS 3.2.5 counts

done!

loading mxe.jc.ijc rMATS 3.2.5 counts

done!

loading mxe.jc.sjc rMATS 3.2.5 counts

done!

loading ri.jc.ijc rMATS 3.2.5 counts

done!

loading ri.jc.sjc rMATS 3.2.5 counts

done!

loading se.jc.ijc rMATS 3.2.5 counts

done!

loading se.jc.sjc rMATS 3.2.5 counts

done!



ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,0,0,2,0
2,0,0,10,0
3,0,0,0,0
4,2,0,0,0
5,1,0,0,0


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,9,151,51,263
2,0,0,0,0
3,2,0,0,0
4,0,0,0,0
5,0,0,0,0


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,0,0,0,2
2,0,0,0,1
3,1,0,0,1
4,14,14,3,9
5,22,67,10,20


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,56,79,30,778
2,0,0,0,0
3,0,0,0,0
4,25,13,12,74
5,0,0,0,0


...

ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,22,36,7,40
2,10,0,4,0
3,1,0,0,0
4,4,0,2,0
5,51,4,8,24


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,0,0,5,0
2,3,0,1,6
3,2,0,0,6
4,9,0,4,15
5,112,64,113,59


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,0,0,0,0
2,0,0,0,0
3,2,0,0,0
4,88,98,25,206
5,70,11,34,37


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,0,0,0,0
2,26,247,103,620
3,1,0,1,0
4,0,1,1,2
5,3,0,2,3


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855
<int>,<int>,<int>,<int>,<int>
1,2,0,1,3
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
5,0,5,3,8


## Read in metadata 

- `Sequence Read Archive (SRA)` Accession Data, `SRR` numbers
- `Genome Tissue Expression (GTEx)` Clinical Annotation

In [6]:
if (!("SraRunTable.txt.gz" %in% list.files("../data/"))) {
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "TheJacksonLaboratory/sbas", 
        file = "SraRunTable.txt.gz",
        tag  = "GTExV8.v1.0", 
        dest = "../data/")
    
    message("Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..\n")   
    metadata <- data.table::fread("../data/SraRunTable.txt.gz")
    message("done!")
} else {
    message("Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..\n")   
    metadata <- data.table::fread("../data/SraRunTable.txt.gz")
    message("done!\n")
}

if (!("gtex.rds" %in% list.files("../data/"))) {
    message("Downloading and loading obj with GTEx v8 with 'yarn::downloadGTExV8()'\n")
    obj <- yarn::downloadGTExV8(type='genes',file='../data/gtex.rds')
    message("Done!\n")

} else {
# Load with readRDS() if gtex.rds available in data/
    message("Loading obj GTEx v8 rds object with readRDS from ../data/gtex.rds ..\n")   
    obj <- readRDS(file = "../data/gtex.rds")
    message("Done!\n")
    message("Generating sha256sum for gtex.rds ..\n")    
    message(system("sha256sum ../data/gtex.rds", intern = TRUE))
    message("Done!\n")
} 
if (! (file.exists("../data/fromGTF.tar.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching fromGTF.tar.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "fromGTF.tar.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_gtf",
                           tag  = "rMATS.3.2.5.gencode.v30",
                           show_progress = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.tar.gz into ../data")
    system("mkdir -p ../data && tar xvfz ../data/fromGTF.tar.gz -C ../data", intern = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.*.txt.gz into ../data")
    system("gunzip  ../data/fromGTF*.txt.gz ", intern = TRUE)
    message("Done!\n")
    message("Reading fromGTF.SE.txt into fromGTF.SE")    
    fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)
    message("Done!\n") 
}

Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..

done!

Loading obj GTEx v8 rds object with readRDS from ../data/gtex.rds ..

Done!

Generating sha256sum for gtex.rds ..

c3c81a2b5b1f17811d2ab828edf1d4c65e8e4a6632964db73555c4b5737fadf0  ../data/gtex.rds
Done!



## Quality control, preprocessing of data 

We observed above that our phenotype data have 2 more observations than our expression data. Let's inspect what these samples are:

In [7]:
tail (pData(obj),2)
dim(pData(obj))
dim(exprs(obj))
dim(obj)
sample_names=as.vector(as.character(colnames(exprs(obj))))
pheno_sample_names=as.vector(as.character(rownames(pData(obj))))
length(pheno_sample_names)
length(sample_names)

if (length(pheno_sample_names) > length(sample_names)) {
    superset <- pheno_sample_names
    subset   <- sample_names    
} 

if (length(pheno_sample_names) < length(sample_names)) {
    superset <- sample_names
    subset   <- pheno_sample_names   
} 

non_overlaps <- setdiff( superset, subset)

message("The non-overlapping IDs between pheno and count data are:\n", 
        paste(non_overlaps, collapse = "\n") )
logical_match_names=superset %in% subset
length(logical_match_names)
table(logical_match_names)
obj        <- obj       [logical_match_names==TRUE,]
pData(obj) <- pData(obj)[logical_match_names==TRUE,]

tail (pData(obj),2)
dim(pData(obj))
dim(exprs(obj))
dim(obj)


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX-YF7O-2326-101833-SM-5CVN9,NA,NA,NA,NA,NA,Skin,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-YEC3-1426-101806-SM-5PNXX,NA,NA,NA,NA,NA,Stomach,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 17384    67

[1] 55878 17382

Features  Samples 
   55878    17382

[1] 17384

...

logical_match_names
FALSE  TRUE 
    2 17382 

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2626-SM-5E45Y,1,B1,"2 pieces, includes internal fat (up to 20%) and tendon",8.50000,Muscle,Muscle - Skeletal,0011907,248,428,⋯,50.6647,0.010466900,0.9956840,NA,0,49.6941,GTEX-ZZPU,2,50-59,0
GTEX-ZZPU-2726-SM-5NQ8O,GTEX-ZZPU-2726-SM-5NQ8O,1,B1,2 pieces,6.80000,Adipose Tissue,Adipose - Subcutaneous,0002190,249,427,⋯,50.3168,0.005856410,0.9958590,NA,0,50.0760,GTEX-ZZPU,2,50-59,0


[1] 17382    67

[1] 55872 17382

Features  Samples 
   55872    17382

## Read in junction Annotation Data

In [8]:
message("Reading fromGTF.A3SS.txt into fromGTF.A3SS")    
fromGTF.A3SS <- read.table("../data/fromGTF.A3SS.txt", header=TRUE)
message("Done!\n")

message("Reading fromGTF.A5SS.txt into fromGTF.A5SS")    
fromGTF.A5SS <- read.table("../data/fromGTF.A5SS.txt", header=TRUE)
message("Done!\n")

message("Reading fromGTF.MXE.txt into fromGTF.MXE")    
fromGTF.MXE <- read.table("../data/fromGTF.MXE.txt", header=TRUE)
message("Done!\n")

message("Reading fromGTF.RI.txt into fromGTF.RI")    
fromGTF.RI <- read.table("../data/fromGTF.RI.txt", header=TRUE)
message("Done!\n")

message("Reading fromGTF.SE.txt into fromGTF.SE")    
fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)
message("Done!\n")


Reading fromGTF.A3SS.txt into fromGTF.A3SS
Done!

Reading fromGTF.A5SS.txt into fromGTF.A5SS
Done!

Reading fromGTF.MXE.txt into fromGTF.MXE
Done!

Reading fromGTF.RI.txt into fromGTF.RI
Done!

Reading fromGTF.SE.txt into fromGTF.SE
Done!



## Preparing our Data
### Aligning Annotations with Run Data and with Count Data

A description of the GTEx V8 release may be found here https://www.gtexportal.org/home/datasets.   To facilitate the analysis, the Quackenbush lab's Joe Paulson's Yarn Package, https://github.com/jnpaulson/yarn, forked and upgraded to GTEx V8 here https://github.com/TheJacksonLaboratory/yarn/tree/annes-changes.  

The sequences we used for our analysis with rMATS are specified by their SRR number and were obtained from dbGaP, with our application to the controlled access human data.  Beginning with these raw unfiltered fastq files, we ran them through the NextFlow workflow rmats-nf https://github.com/lifebit-ai/rmats-nf. The accession data was downloaded with a repository key from the google cloud bucket (g3).

- `obj`      : GTEx V8 expressionSet 
- `metadata` : SraRunTable.txt holds the SRR accession numbers used to extract samples from dbGaP SRA
- `ijc`      : inclusion junction counts as reported by `rMATS 3.2.5`
- `sjc`      : skipped junction counts as reported by `rMATS 3.2.5`  

For each of the splicing events:

- `a3ss`     : alternative 3` splice site
- `a5ss`     : alternative 5` splice site
- `mxe`      : mutually exclusive exon
- `ri`       : retention intron
- `se`       : skipped exon

In `rMATS 3.2.5`, each junction is encoded in a unique `ID` that is immutable regardless of comparison.  
This `ID` is different for each of the differing splicing events.

### Move Junction ID to Row definition

In [9]:
rownames(a3ss.jc.ijc) <- a3ss.jc.ijc$ID
rownames(a3ss.jc.sjc) <- a3ss.jc.sjc$ID
a3ss.jc.ijc <- a3ss.jc.ijc[,-1]
a3ss.jc.sjc <- a3ss.jc.sjc[,-1]

rownames(a5ss.jc.ijc) <- a3ss.jc.ijc$ID
rownames(a5ss.jc.sjc) <- a3ss.jc.sjc$ID
a5ss.jc.ijc <- a5ss.jc.ijc[,-1]
a5ss.jc.sjc <- a5ss.jc.sjc[,-1]

rownames(mxe.jc.ijc)  <- mxe.jc.ijc$ID
rownames(mxe.jc.sjc)  <- mxe.jc.sjc$ID
mxe.jc.ijc <- mxe.jc.ijc[,-1]
mxe.jc.sjc <- mxe.jc.sjc[,-1]

rownames(ri.jc.ijc)   <- ri.jc.ijc$ID
rownames(ri.jc.sjc)   <- ri.jc.sjc$ID
ri.jc.ijc <- ri.jc.ijc[,-1]
ri.jc.sjc <- ri.jc.sjc[,-1]

rownames(se.jc.ijc)   <- se.jc.ijc$ID
rownames(se.jc.sjc)   <- se.jc.sjc$ID
se.jc.ijc <- se.jc.ijc[,-1]
se.jc.sjc <- se.jc.sjc[,-1]

dim(a3ss.jc.ijc)
dim(a3ss.jc.sjc)
dim(a5ss.jc.ijc)
dim(a5ss.jc.sjc)
dim(mxe.jc.ijc)
dim(mxe.jc.sjc)
dim(ri.jc.ijc)
dim(ri.jc.sjc)
dim(se.jc.ijc)
dim(se.jc.sjc)

a3ss.jc.ijc[1:5,1:5]
a3ss.jc.sjc[1:5,1:5]

a5ss.jc.ijc[1:5,1:5]
a5ss.jc.sjc[1:5,1:5]

mxe.jc.ijc[1:5,1:5]
mxe.jc.sjc[1:5,1:5]

ri.jc.ijc[1:5,1:5]
ri.jc.sjc[1:5,1:5]

se.jc.ijc[1:5,1:5]
se.jc.sjc[1:5,1:5]



[1] 8920 8673

[1] 8920 8673

[1] 5584 8673

[1] 5584 8673

[1] 2979 8673

...

SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880
<int>,<int>,<int>,<int>,<int>
22,36,7,40,10
10,0,4,0,1
1,0,0,0,0
4,0,2,0,1
51,4,8,24,2


SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880
<int>,<int>,<int>,<int>,<int>
0,0,5,0,0
3,0,1,6,1
2,0,0,6,2
9,0,4,15,7
112,64,113,59,43


SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880
<int>,<int>,<int>,<int>,<int>
0,0,0,0,0
0,0,0,0,0
2,0,0,0,0
88,98,25,206,150
70,11,34,37,56


SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880
<int>,<int>,<int>,<int>,<int>
0,0,0,0,1
26,247,103,620,494
1,0,1,0,0
0,1,1,2,0
3,0,2,3,6


SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880
<int>,<int>,<int>,<int>,<int>
2,0,1,3,6
0,0,0,1,0
0,0,0,1,0
0,0,0,1,0
0,5,3,8,4


### Synchronize Clinical Annotations and Accession Run, reducing to only tissues of interest

Join the yarn metadata with the metadata we have (there are redundant samples that have been sequenced multiple times). We want to be sure that we can obtain all required Clinical Annotation information from the YARN GTEx Annotation information, as the SRA metadata is not as reliable.    This will be a one-to-many mapping, as there are multiple sequence runs per 69 samples -- expanding our data set.  There are only a handful of annotations we require: SEX, AGE, DTHHRDY (which is cause of death), SMCENTER.

Note that the numbers in specific age groups expand because of the one to many relationship from sample to sequencing runs. 

#### Using results from analysis of number of samples stored in `tissues.tsv` we keep only those that are members of this reduced tissue list.

In [46]:
# read in all requirements so that the stage is properly set -- 
# if it is clear here -- it will remain clear for the rest of the time
# tissues.tsv contains the subset of files desired for analysis.
tissue_reduction <- read.table(file="../assets/tissues.tsv", header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
colnames(tissue_reduction)  <- c("SMTSD","female","male","include","display_name")

message("\nsize tissue_reduction\n",
        paste(dim(tissue_reduction), collapse=" "))
message("\nsize obj\n",
        paste(dim(obj)), collapse="\n")
message("\nsize pData(obj)\n",
        paste(dim(pData(obj)), collapse=" "))

# only include those tissues we wish to continue with
table(tissue_reduction$include)
tissue_reduction <- tissue_reduction[tissue_reduction$include==1,]

# test to make sure we don't have nonsense
keep <- (pData(obj)$SMTSD== "breast_mammary_tissue")
message("\nTEST: how many to keep of obj in breast_mammary_tissue\n",
        paste(table(keep), collapse = " "))
tobj        = obj      [,keep]
pData(tobj) = pData(obj[,keep])
message("\nTEST: size breast_mammary_tissue obj:tobj\n",
        paste(dim(tobj), collapse=" "))
message("\nTEST: size phenotype object pData(tobj)\n",
        paste(dim(pData(tobj)), collapse=" "))
pData(tobj)[1,]
rm(keep)
# end test

# create a matching tissue name to go with the expressionSet phenotype object
pData(obj)$SMTSD       <- factor(snakecase::to_snake_case(as.character(pData(obj)$SMTSD)))
tissue_reduction$SMTSD <- factor(snakecase::to_snake_case(as.character(tissue_reduction$SMTSD)))

message("\nlength tissues in phenotype data\n",
        paste(length(levels(pData(obj)$SMTSD)), collapse = " "))
message("\nlength tissues in tissue_reduction data\n",
        paste(length(tissue_reduction$SMTSD), collapse = " "))

keep <- pData(obj)$SMTSD %in% tissue_reduction$SMTSD
message("\nlength tissue in samples phenotype data\n",
        paste(length(pData(obj)$SMTSD), collapse = " "))
message("\nlength keep obj \n",
        paste(length(keep), collapse = " "))
message("\nhow many to keep in phenotype data\n",
        paste(table(keep), collapse = " "))

# both obj and pData(obj) need to be adjusted
reduced_obj        <- obj       [          ,keep==TRUE]
pData(reduced_obj) <- pData(obj)[keep==TRUE,          ]
rm(keep)
message("\nsize reduced_obj\n",
        paste(dim(reduced_obj)), collapse=" ")
message("\nsize pData(reduced_obj)\n",
        paste(dim(pData(reduced_obj)), collapse=" "))
message("\nlength tissues in phenotype data\n",
        paste(length(levels(pData(reduced_obj)$SMTSD)), collapse = " "))

# test to make sure we don't have nonsense
keep = pData(reduced_obj)$SMTSD== "breast_mammary_tissue"
message("\nTEST: how many to keep in to have only breast_mammary_tissue\n",
        paste(table(keep), collapse = " "))
tobj        = reduced_obj       [          ,keep==TRUE]
pData(tobj) = pData(reduced_obj)[keep==TRUE,          ]
message("\nTEST: size breast_mammary_tissue obj:tobj\n",
        paste(dim(tobj), collapse=" "))
message("\nTEST: size phenotype object pData(tobj)\n",
        paste(dim(pData(tobj)), collapse=" "))
pData(tobj)[1,]
rm(keep)
# end test


size tissue_reduction
50 5

size obj
5587210372


size pData(obj)
10372 68



 0  1 
11 39 


TEST: how many to keep of obj in breast_mammary_tissue
10086 286

TEST: size breast_mammary_tissue obj:tobj
55872 286

TEST: size phenotype object pData(tobj)
286 68


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,tissue
,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX.1117F.2826.SM.5GZXL,GTEX.1117F.2826.SM.5GZXL,1,B1,"2 pieces, fibrocystic changes, rep ductal/lobular elements delineated",5.80000,Breast,breast_mammary_tissue,0008367,1340,1008,⋯,0.015023200,0.9943150,NA,0,50.0068,GTEX-1117F,2,60-69,4,breast_mammary_tissue



length tissues in phenotype data
39

length tissues in tissue_reduction data
39

length tissue in samples phenotype data
10372

length keep obj 
10372

how many to keep in phenotype data
10372

size reduced_obj
5587210372 

size pData(reduced_obj)
10372 68

length tissues in phenotype data
39

TEST: how many to keep in to have only breast_mammary_tissue
10086 286

TEST: size breast_mammary_tissue obj:tobj
55872 286

TEST: size phenotype object pData(tobj)
286 68


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,tissue
,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX.1117F.2826.SM.5GZXL,GTEX.1117F.2826.SM.5GZXL,1,B1,"2 pieces, fibrocystic changes, rep ductal/lobular elements delineated",5.80000,Breast,breast_mammary_tissue,0008367,1340,1008,⋯,0.015023200,0.9943150,NA,0,50.0068,GTEX-1117F,2,60-69,4,breast_mammary_tissue


#### Synchronize accession metadata and phenotype data
A kind of transitive closure.   Metadata links the count data to the phenotype data.
Begin with synchronizing accession metadata and phenotype data - which has been reduced - `reduced_obj` inputs here 

In [47]:
# let's limit the phenotype object and then align the metadata file
metadata <- data.table::fread("../data/SraRunTable.txt.gz")
metadata$SAMPID           <- gsub('-','\\.',metadata$'Sample Name')
pData(reduced_obj)$SAMPID <- gsub('-','\\.',pData(reduced_obj)$SAMPID)

message("\nsize accession SraRunTable \n",
        paste(dim(metadata), collapse=" "))
message("\nsize reduced_obj\n",
        paste(dim(reduced_obj), collapse=" "))
message("\nsize pData(reduced_obj)\n",
        paste(dim(pData(reduced_obj)), collapse=" "))
rownames(pData(reduced_obj))<- pData(reduced_obj)$SAMPID

# keep only those runs (as epitomized by the metadata_samples) in the phenotype set
metadata_samples   <- as.character(metadata$SAMPID)
phenotype_samples  <- as.character(pData(reduced_obj)$SAMPID)

# any undefined (N/A) sample names? These results will be zero
message("\n any undefined (NA) sample names?\n",
        paste(sum(is.na(metadata_samples)), collapse=" "))
message("\n any undefined (NA) sample names?\n",
        paste(sum(is.na(phenotype_samples)), collapse=" "))

keep <- phenotype_samples %in% metadata_samples
message("\nlength keep metadata samples in phenotype samples \n",
        paste(length(keep), collapse = " "))
message("\nhow many to keep in phenotype obj\n",
        paste(table(keep), collapse = " "))

reduced_obj2        <- reduced_obj       [          ,keep==TRUE]
pData(reduced_obj2) <- pData(reduced_obj)[keep==TRUE,          ]
message("\nsize reduced_obj2\n",
        paste(dim(reduced_obj2), collapse=" "))
message("\nsize pData(reduced_obj2)\n",
        paste(dim(pData(reduced_obj2)), collapse=" "))

# test to make sure we don't have nonsense
keep = pData(reduced_obj2)$SMTSD == "breast_mammary_tissue"
message("\nTEST: how many to keep in to have only breast_mammary_tissue\n",
        paste(table(keep), collapse = " "))
tobj        = reduced_obj2       [          ,keep==TRUE]
pData(tobj) = pData(reduced_obj2)[keep==TRUE,          ]
message("\nTEST: size breast_mammary_tissue obj:tobj\n",
        paste(dim(tobj), collapse=" "))
message("\nTEST: size phenotype object pData(tobj)\n",
        paste(dim(pData(tobj)), collapse=" "))
pData(tobj)[1,]
rm(keep)
# end test

# now go the other way - make sure the metadata samples are in sync with the phenotype samples
# note that we are now with `reduced_obj2`
metadata_samples   <- as.character(metadata$SAMPID)
phenotype_samples  <- as.character(pData(reduced_obj2)$SAMPID)
message("\nlength of metadata_samples\n",
        paste(length(metadata_samples), collapse=" "))
message("\nlength of phenotype_samples\n",
        paste(length(phenotype_samples), collapse=" "))
message("\ndimension of metadata\n",
        paste(dim(metadata), collapse=" "))
keep <- metadata_samples %in% phenotype_samples
message("\nlength keep phenotype samples in metadata samples \n",
        paste(length(keep), collapse = " "))
message("\nhow many to keep in metadata samples\n")

reduced_metadata <- metadata[keep==TRUE,]
message("\ndimension of reduced_metadata\n",
        paste(dim(reduced_metadata), collapse=" "))
rm(keep)

# test to make sure we don't have nonsense
keep = pData(reduced_obj2)$SMTSD== "breast_mammary_tissue"
message("\nTEST: how many to keep in to have only breast_mammary_tissue\n",
        paste(table(keep), collapse = " "))
tobj        = reduced_obj2       [          ,keep==TRUE]
pData(tobj) = pData(reduced_obj2)[keep==TRUE,          ]
message("\nTEST: size breast_mammary_tissue obj:tobj\n",
        paste(dim(tobj), collapse=" "))
message("\nTEST: size phenotype object pData(tobj)\n",
        paste(dim(pData(tobj)), collapse=" "))
pData(tobj)[1,]
rm(keep)

breast_metadata_samples   <- as.character(reduced_metadata$SAMPID)
breast_phenotype_samples  <- as.character(pData(tobj)$SAMPID)
keep = breast_metadata_samples %in% breast_phenotype_samples
table(keep)
message("\nTEST: how many breast_mammary_tissue samples in reduced_metadata\n",
        paste(table(keep), collapse = " "))
breast_samples <- reduced_metadata[keep,]
message("\nTEST: number reduced_metadata breast_samples\n",
        paste(length(breast_samples$SAMPID), collapse=" "))
message("\nTEST: number phenotype obj breast_samples\n",
        paste(length(pData(tobj)$SAMPID), collapse=" "))
breast_samples[1,]
# end test



size accession SraRunTable 
24667 80

size reduced_obj
55872 10372

size pData(reduced_obj)
10372 68

 any undefined (NA) sample names?
0

 any undefined (NA) sample names?
0

length keep metadata samples in phenotype samples 
10372

how many to keep in phenotype obj
10372

size reduced_obj2
55872 10372

size pData(reduced_obj2)
10372 68

TEST: how many to keep in to have only breast_mammary_tissue
10086 286

TEST: size breast_mammary_tissue obj:tobj
55872 286

TEST: size phenotype object pData(tobj)
286 68


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,tissue
,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX.1117F.2826.SM.5GZXL,GTEX.1117F.2826.SM.5GZXL,1,B1,"2 pieces, fibrocystic changes, rep ductal/lobular elements delineated",5.80000,Breast,breast_mammary_tissue,0008367,1340,1008,⋯,0.015023200,0.9943150,NA,0,50.0068,GTEX-1117F,2,60-69,4,breast_mammary_tissue



length of metadata_samples
24667

length of phenotype_samples
10372

dimension of metadata
24667 80

length keep phenotype samples in metadata samples 
24667

how many to keep in metadata samples


dimension of reduced_metadata
18214 80

TEST: how many to keep in to have only breast_mammary_tissue
10086 286

TEST: size breast_mammary_tissue obj:tobj
55872 286

TEST: size phenotype object pData(tobj)
286 68


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,tissue
,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
GTEX.1117F.2826.SM.5GZXL,GTEX.1117F.2826.SM.5GZXL,1,B1,"2 pieces, fibrocystic changes, rep ductal/lobular elements delineated",5.80000,Breast,breast_mammary_tissue,0008367,1340,1008,⋯,0.015023200,0.9943150,NA,0,50.0068,GTEX-1117F,2,60-69,4,breast_mammary_tissue


keep
FALSE  TRUE 
17722   492 


TEST: how many breast_mammary_tissue samples in reduced_metadata
17722 492

TEST: number reduced_metadata breast_samples
492

TEST: number phenotype obj breast_samples
286


Run,analyte_type,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,body_site,⋯,product_part_number (exp),product_part_number (run),sample_barcode (exp),sample_barcode (run),is_technical_control,target_set (exp),primary_disease (exp),secondary_accessions (run),Alignment_Provider (run),SAMPID
<chr>,<chr>,<chr>,<int>,<int64>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SRR821498,RNA:Total RNA,RNA-Seq,152,7879119272,PRJNA75899,SAMN01994192,GTEx,GTEX-WI4N-1426-SM-3LK7H,Breast - Mammary Tissue,⋯,,,NA,NA,,,,,,GTEX.WI4N.1426.SM.3LK7H


###  Limit obj and metadata to size of the ijc and sjc columns 

Things are complicated and can easily lead to data disaster.
We need to go forward and backwards with these data - and see how to have them arranged.  
Things we know to be true:
1. Run numbers are unique
2. Samples (which are Donor and Tissue combined) have been sequenced more than once.

We need to keep this clear as the metadata are used to sort out tissues, sex, and other factors - and these are based upon a Donor.  Relationships are as follows:

* One Donor can have many tissues (one donor has as many as 54 tissues)
* One Tissue may have many sequencing runs (One Tissue has been as of this writing sequenced as many as 3 times)

The analyst must keep this information in mind as manipulations occur.

Here we use `se.jc.ijc` as exemplar as well as model for all subsequent matrices.  Arrangements of `se.jc.ijc` will ripple to all other matrices because sample order is the same between all samples.

Entering into this step we have for our phenotype object:
1. reduced_obj2
2. reduced_metadata

In [48]:
metadata_for_counts = colnames(se.jc.ijc)%in% reduced_metadata$Run
counts_for_metadata = reduced_metadata$Run %in% colnames(se.jc.ijc)
table(metadata_for_counts)
table(counts_for_metadata)

counts          <- data.matrix(se.jc.ijc)
counts_metadata <- reduced_metadata
dim(counts)
dim(counts_metadata)

counts          <- counts         [                         ,metadata_for_counts==TRUE]
counts_metadata <- counts_metadata[counts_for_metadata==TRUE,                         ]

# now we arrange metadata and counts to be in the same order
# Run numbers are unique, but we will replace the columns with SAMPID as metadata is by SAMPID
i = order(colnames(counts))
j = order(counts_metadata$Run)

ose.jc.ijc       = counts[,i]
ocounts_metadata = counts_metadata[j,]

tail(colnames(ose.jc.ijc),3)
tail(ocounts_metadata$Run,3)

colnames(ose.jc.ijc) = as.character(ocounts_metadata$SAMPID)
tail(colnames(ose.jc.ijc),3)
tail(ocounts_metadata$SAMPID,3)
sum(table(colnames(ose.jc.ijc))==1)
sum(table(colnames(ose.jc.ijc))==2)
sum(table(colnames(ose.jc.ijc))==3)
sum(table(colnames(ose.jc.ijc))==4)
dim(ose.jc.ijc)

metadata_for_counts
FALSE  TRUE 
 1653  7020 

counts_for_metadata
FALSE  TRUE 
11194  7020 

[1] 42611  8673

[1] 18214    80

[1] "SRR821653" "SRR821690" "SRR821715"

...

[1] 6891

[1] 15

[1] 33

[1] 0

[1] 42611  7020

## Now extend this organization to all count matrices
now that the data are organized and accounted for, extend to the rest of the splicing factors and their count matrices

In [49]:
oa3ss.jc.ijc  <- data.matrix(a3ss.jc.ijc)
oa3ss.jc.sjc  <- data.matrix(a3ss.jc.sjc)
oa3ss.jc.ijc  <- oa3ss.jc.ijc[,metadata_for_counts==TRUE]
oa3ss.jc.sjc  <- oa3ss.jc.sjc[,metadata_for_counts==TRUE]

oa5ss.jc.ijc  <- data.matrix(a5ss.jc.ijc)
oa5ss.jc.sjc  <- data.matrix(a5ss.jc.sjc)
oa5ss.jc.ijc  <- oa5ss.jc.ijc[,metadata_for_counts==TRUE]
oa5ss.jc.sjc  <- oa5ss.jc.sjc[,metadata_for_counts==TRUE]

omxe.jc.ijc   <- data.matrix(mxe.jc.ijc)
omxe.jc.sjc   <- data.matrix(mxe.jc.sjc)
omxe.jc.ijc   <- omxe.jc.ijc[,metadata_for_counts==TRUE]
omxe.jc.sjc   <- omxe.jc.sjc[,metadata_for_counts==TRUE]

ori.jc.ijc    <- data.matrix(ri.jc.ijc)
ori.jc.sjc    <- data.matrix(ri.jc.sjc)
ori.jc.ijc    <- ori.jc.ijc[,metadata_for_counts==TRUE]
ori.jc.sjc    <- ori.jc.sjc[,metadata_for_counts==TRUE]

ose.jc.ijc    <- data.matrix(se.jc.ijc)
ose.jc.sjc    <- data.matrix(se.jc.sjc)
ose.jc.ijc    <- ose.jc.ijc[,metadata_for_counts==TRUE]
ose.jc.sjc    <- ose.jc.sjc[,metadata_for_counts==TRUE]

oa3ss.jc.ijc  <- oa3ss.jc.ijc[,i]
oa3ss.jc.sjc  <- oa3ss.jc.sjc[,i]

oa5ss.jc.ijc  <- oa5ss.jc.ijc[,i]
oa5ss.jc.sjc  <- oa5ss.jc.sjc[,i]

omxe.jc.ijc   <- omxe.jc.ijc[,i]
omxe.jc.sjc   <- omxe.jc.sjc[,i]

ori.jc.ijc    <- ori.jc.ijc[,i]
ori.jc.sjc    <- ori.jc.sjc[,i]

ose.jc.ijc    <- ose.jc.ijc  [,i]
ose.jc.sjc    <- ose.jc.sjc  [,i]

oa3ss.jc.ijc[1:5,1:5]
oa3ss.jc.sjc[1:5,1:5]

oa5ss.jc.ijc[1:5,1:5]
oa5ss.jc.sjc[1:5,1:5]

omxe.jc.ijc[1:5,1:5]
omxe.jc.sjc[1:5,1:5]

ori.jc.ijc[1:5,1:5]
ori.jc.sjc[1:5,1:5]

ose.jc.ijc[1:5,1:5]
ose.jc.sjc[1:5,1:5]

colnames(oa3ss.jc.ijc) = as.character(ocounts_metadata$SAMPID)
colnames(oa3ss.jc.sjc) = as.character(ocounts_metadata$SAMPID)

colnames(oa5ss.jc.ijc) = as.character(ocounts_metadata$SAMPID)
colnames(oa5ss.jc.sjc) = as.character(ocounts_metadata$SAMPID)

colnames(omxe.jc.ijc) = as.character(ocounts_metadata$SAMPID)
colnames(omxe.jc.sjc) = as.character(ocounts_metadata$SAMPID)

colnames(ori.jc.ijc) = as.character(ocounts_metadata$SAMPID)
colnames(ori.jc.sjc) = as.character(ocounts_metadata$SAMPID)

colnames(ose.jc.ijc) = as.character(ocounts_metadata$SAMPID)
colnames(ose.jc.sjc) = as.character(ocounts_metadata$SAMPID)

tail(colnames(oa3ss.jc.ijc),3)
tail(colnames(oa3ss.jc.sjc),3)

tail(colnames(oa5ss.jc.ijc),3)
tail(colnames(oa5ss.jc.sjc),3)

tail(colnames(omxe.jc.ijc),3)
tail(colnames(omxe.jc.sjc),3)

tail(colnames(ori.jc.ijc),3)
tail(colnames(ori.jc.sjc),3)

tail(colnames(ose.jc.ijc),3)
tail(colnames(ose.jc.sjc),3)


SRR1068808,SRR1068832,SRR1068880,SRR1068929,SRR1068953
0,2,2,0,0
0,10,0,4,4
0,0,1,0,1
0,0,2,0,1
0,0,1,0,1


SRR1068808,SRR1068832,SRR1068880,SRR1068929,SRR1068953
151,51,292,59,91
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0


SRR1068808,SRR1068832,SRR1068880,SRR1068929,SRR1068953
0,0,1,2,0
0,0,0,0,0
0,0,0,1,0
14,3,4,3,11
67,10,167,26,31


SRR1068808,SRR1068832,SRR1068880,SRR1068929,SRR1068953
79,30,293,120,107
0,0,0,0,0
0,0,0,0,0
13,12,10,8,18
0,0,0,0,0


SRR1068808,SRR1068832,SRR1068880,SRR1068929,SRR1068953
5,3,4,1,4
0,0,0,1,0
0,0,0,0,0
1,0,0,3,2
0,0,0,7,32


...

[1] "GTEX.WOFL.0006.SM.3TW8K"     "GTEX.NPJ8.0011.R1a.SM.33HCB"
[3] "GTEX.OHPK.0226.SM.3MJH6"

[1] "GTEX.WOFL.0006.SM.3TW8K"     "GTEX.NPJ8.0011.R1a.SM.33HCB"
[3] "GTEX.OHPK.0226.SM.3MJH6"

[1] "GTEX.WOFL.0006.SM.3TW8K"     "GTEX.NPJ8.0011.R1a.SM.33HCB"
[3] "GTEX.OHPK.0226.SM.3MJH6"

[1] "GTEX.WOFL.0006.SM.3TW8K"     "GTEX.NPJ8.0011.R1a.SM.33HCB"
[3] "GTEX.OHPK.0226.SM.3MJH6"

[1] "GTEX.WOFL.0006.SM.3TW8K"     "GTEX.NPJ8.0011.R1a.SM.33HCB"
[3] "GTEX.OHPK.0226.SM.3MJH6"

### Sanity Check

Spot check one tissue and make sure there isn't nonsense
Our current matrices are:
1. ocounts_metadata for the connection from counts to phenotype via the Run and SAMPID
2. pData(reduced_obj2)

In [50]:
# test to make sure we don't have nonsense
rm(keep)
keep = pData(reduced_obj2)$SMTSD == "breast_mammary_tissue"
table(keep)
tobj = reduced_obj2[,keep]
dim(tobj)
dim(pData(tobj))
breast_metadata_samples   <- as.character(ocounts_metadata$SAMPID)
breast_phenotype_samples  <- as.character(pData(tobj)$SAMPID)
rm(keep)
keep = breast_metadata_samples %in% breast_phenotype_samples
table(keep)
breast_samples <- ocounts_metadata[keep,]
length(breast_samples$SAMPID)
length(pData(tobj)$SAMPID)
breast_samples[1,]
# end test

keep
FALSE  TRUE 
10086   286 

Features  Samples 
   55872      286

[1] 286  68

keep
FALSE  TRUE 
 6842   178 

[1] 178

[1] 286

Run,analyte_type,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,body_site,⋯,product_part_number (exp),product_part_number (run),sample_barcode (exp),sample_barcode (run),is_technical_control,target_set (exp),primary_disease (exp),secondary_accessions (run),Alignment_Provider (run),SAMPID
<chr>,<chr>,<chr>,<int>,<int64>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SRR1068977,RNA:Total RNA,RNA-Seq,152,6706681864,PRJNA75899,SAMN02465522,GTEx,GTEX-XBED-1626-SM-47JYN,Breast - Mammary Tissue,⋯,P-RNA-0002,P-RNA-0002,NA,NA,,,,,,GTEX.XBED.1626.SM.47JYN


### which metadata shall we use?
There is a lack of correspondeence between the metadata in the Sequence Read Archive table as obtained from dbGaP SRR Run Explorer and the GTEx V8 Release.   Not sure which one to use!  

In [51]:
table(pData(reduced_obj2)$AGE)
table(pData(reduced_obj2)$DTHHRDY)
table(pData(reduced_obj2)$SMCENTER)
table(pData(reduced_obj2)$SEX)
table(ocounts_metadata$sex)


20-29 30-39 40-49 50-59 60-69 70-79 
  760   771  1729  3484  3312   316 


   0    1    2    3    4 
5523  467 2738  466 1082 


        B1     B1, A1         C1     C1, A1 C1, B1, A1         D1     D1, A1 
      6154        377       3110        603          3         70         55 


   1    2 
6796 3576 


female   male 
  2501   4519 

#### Exploring the ijc and sjc Count data 

For each sample, we have ijc and sjc count data, for each alternative splicing type, we have the following number of alternative splicing events:

* `A3SS`: ` 8,920`
* `A5SS`: ` 5,584`
* `MXE `: ` 2,979`
* `RI`  : ` 6,312`
* `SE  `: `42,611`

For exon skipping events (SE), we have 42,611 non-zero junction IDs the (first dimension of the ijc and sjc cout table) for the skipped exon event for breast-Mammary Tissue, 191 individuals.  These are healthy individuals, and we are studying the impact of sex on the occurrence or non-occurance of specific alternative splicing events.   We explore the information we ahve about these junctions and create a construct, as_event, which accounts for the junction under exploration.

The `IJC` and `SJC` counts are in many ways two sides of the same coin.  Both our the observational output and we wish to see how robust each are in their ability to separate out the samples to provide for us differentially expressed isoform events as measured by their counts.   Each junction is in a manner a specific marker to specific isoform events that may or may not be shared between the genders.   If there is significant results, then this is indicative of the separation achieved by isoform specific differentiation.   In our model we will use these in combination, it is important to see if they will yield the results we are looking for.


### Reduce count matrices based upon the reduced tissue information - count matrix by column adjustment

We wish to keep only those samples (columns) in our count data that are a member of the reduced tissue set.

In [52]:
metadata_names <- as.character(ocounts_metadata$SAMPID)
count_names    <- as.character(colnames(ose.jc.ijc))

keep  <- count_names %in% metadata_names

table(keep)
length(keep)
length(metadata_names)
length(count_names)

a3ss.jc.ijc2 = oa3ss.jc.ijc[,keep==TRUE] 
a3ss.jc.sjc2 = oa3ss.jc.sjc[,keep==TRUE] 

a5ss.jc.ijc2 = oa5ss.jc.ijc[,keep==TRUE] 
a5ss.jc.sjc2 = oa5ss.jc.sjc[,keep==TRUE] 

mxe.jc.ijc2 = omxe.jc.ijc[,keep==TRUE] 
mxe.jc.sjc2 = omxe.jc.sjc[,keep==TRUE] 

ri.jc.ijc2 = ori.jc.ijc[,keep==TRUE] 
ri.jc.sjc2 = ori.jc.sjc[,keep==TRUE] 

se.jc.ijc2 = ose.jc.ijc[,keep==TRUE] 
se.jc.sjc2 = ose.jc.sjc[,keep==TRUE] 

dim(a3ss.jc.ijc2)
dim(a3ss.jc.sjc2)
dim(a5ss.jc.ijc2)
dim(a5ss.jc.sjc2)
dim(mxe.jc.ijc2)
dim(mxe.jc.sjc2)
dim(ri.jc.ijc2)
dim(ri.jc.sjc2)
dim(se.jc.ijc2)
dim(se.jc.sjc2)

a3ss.jc.ijc2[1:5,1:5]
a3ss.jc.sjc2[1:5,1:5]

a5ss.jc.ijc2[1:5,1:5]
a5ss.jc.sjc2[1:5,1:5]

mxe.jc.ijc2[1:5,1:5]
mxe.jc.sjc2[1:5,1:5]

ri.jc.ijc2[1:5,1:5]
ri.jc.sjc2[1:5,1:5]

se.jc.ijc2[1:5,1:5]
se.jc.sjc2[1:5,1:5]


keep
TRUE 
7020 

[1] 7020

[1] 7020

[1] 7020

[1] 8920 7020

...

GTEX.QV44.0008.SM.447AX,GTEX.UJMC.0426.SM.4IHJF,GTEX.O5YT.0008.SM.4E3IQ,GTEX.PWN1.0726.SM.48TDR,GTEX.PWN1.1526.SM.48TDA
36,7,10,15,1
0,4,1,2,2
0,0,0,0,0
0,2,1,0,0
4,8,2,25,74


GTEX.QV44.0008.SM.447AX,GTEX.UJMC.0426.SM.4IHJF,GTEX.O5YT.0008.SM.4E3IQ,GTEX.PWN1.0726.SM.48TDR,GTEX.PWN1.1526.SM.48TDA
0,5,0,1,1
0,1,1,0,0
0,0,2,0,1
0,4,7,4,2
64,113,43,40,247


GTEX.QV44.0008.SM.447AX,GTEX.UJMC.0426.SM.4IHJF,GTEX.O5YT.0008.SM.4E3IQ,GTEX.PWN1.0726.SM.48TDR,GTEX.PWN1.1526.SM.48TDA
0,0,0,1,1
0,0,0,0,0
0,0,0,0,0
98,25,150,64,37
11,34,56,38,125


GTEX.QV44.0008.SM.447AX,GTEX.UJMC.0426.SM.4IHJF,GTEX.O5YT.0008.SM.4E3IQ,GTEX.PWN1.0726.SM.48TDR,GTEX.PWN1.1526.SM.48TDA
0,0,1,0,0
247,103,494,145,145
0,1,0,0,1
1,1,0,0,1
0,2,6,1,1


GTEX.QV44.0008.SM.447AX,GTEX.UJMC.0426.SM.4IHJF,GTEX.O5YT.0008.SM.4E3IQ,GTEX.PWN1.0726.SM.48TDR,GTEX.PWN1.1526.SM.48TDA
0,1,6,1,0
0,0,0,0,1
0,0,0,0,0
0,0,0,0,0
5,3,4,0,3


### Keeping only chromosomes shared male female - count matrix by junction row adjustment 

The Y chromosome spans more than 59 million base pairs of DNA and represents almost 2 percent of the total DNA in cells. Each person normally has one pair of sex chromosomes in each cell. The Y chromosome is present in males, who have one X and one Y chromosome, while females have two X chromosomes. Since our analysis is on the comparative differences, we must eliminate chrY from our analyses.

To do so, we grab the annotation from the GTF file and remove those junctions that correspond to the genes on this chromosome.

So this is a count matrix row adjustment.

In [53]:
dim(fromGTF.A3SS)
dim(fromGTF.A5SS)
dim(fromGTF.MXE)
dim(fromGTF.RI)
dim(fromGTF.SE)

A3SS.genes <- factor(fromGTF.A3SS$geneSymbol)
A5SS.genes <- factor(fromGTF.A5SS$geneSymbol)
MXE.genes  <- factor(fromGTF.MXE$geneSymbol)
RI.genes   <- factor(fromGTF.RI$geneSymbol)
SE.genes   <- factor(fromGTF.SE$geneSymbol)

length(levels(A3SS.genes))    
length(levels(A5SS.genes))    
length(levels(MXE.genes))    
length(levels(RI.genes))    
length(levels(SE.genes))    

table(fromGTF.A3SS$chr)
table(fromGTF.A5SS$chr)
table(fromGTF.MXE$chr)
table(fromGTF.RI$chr)
table(fromGTF.SE$chr)

A3SS.keepAllJunctionsButChrY <- (fromGTF.A3SS$chr != "chrY")
A5SS.keepAllJunctionsButChrY <- (fromGTF.A5SS$chr != "chrY")
MXE.keepAllJunctionsButChrY  <- (fromGTF.MXE$chr  != "chrY")
RI.keepAllJunctionsButChrY   <- (fromGTF.RI$chr   != "chrY")
SE.keepAllJunctionsButChrY   <- (fromGTF.SE$chr   != "chrY")

table(A3SS.keepAllJunctionsButChrY)
sum(table(A3SS.keepAllJunctionsButChrY))

table(A5SS.keepAllJunctionsButChrY)
sum(table(A5SS.keepAllJunctionsButChrY))

table(MXE.keepAllJunctionsButChrY)
sum(table(MXE.keepAllJunctionsButChrY))

table(RI.keepAllJunctionsButChrY)
sum(table(RI.keepAllJunctionsButChrY))

table(SE.keepAllJunctionsButChrY)
sum(table(SE.keepAllJunctionsButChrY))

a3ss_fromGTF_no_chrY          <- fromGTF.A3SS[A3SS.keepAllJunctionsButChrY,]
a3ss_jc_ijc_no_chrY           <- a3ss.jc.ijc2[A3SS.keepAllJunctionsButChrY,]
a3ss_jc_sjc_no_chrY           <- a3ss.jc.sjc2[A3SS.keepAllJunctionsButChrY,]
rownames(a3ss_jc_ijc_no_chrY) <- rownames(a3ss_fromGTF_no_chrY)
rownames(a3ss_jc_sjc_no_chrY) <- rownames(a3ss_fromGTF_no_chrY)

a5ss_fromGTF_no_chrY          <- fromGTF.A5SS[A5SS.keepAllJunctionsButChrY,]
a5ss_jc_ijc_no_chrY           <- a5ss.jc.ijc2[A5SS.keepAllJunctionsButChrY,]
a5ss_jc_sjc_no_chrY           <- a5ss.jc.sjc2[A5SS.keepAllJunctionsButChrY,]
rownames(a5ss_jc_ijc_no_chrY) <- rownames(a5ss_fromGTF_no_chrY)
rownames(a5ss_jc_sjc_no_chrY) <- rownames(a5ss_fromGTF_no_chrY)

mxe_fromGTF_no_chrY          <- fromGTF.MXE[MXE.keepAllJunctionsButChrY,]
mxe_jc_ijc_no_chrY           <- mxe.jc.ijc2[MXE.keepAllJunctionsButChrY,]
mxe_jc_sjc_no_chrY           <- mxe.jc.sjc2[MXE.keepAllJunctionsButChrY,]
rownames(mxe_jc_ijc_no_chrY) <- rownames(mxe_fromGTF_no_chrY)
rownames(mxe_jc_sjc_no_chrY) <- rownames(mxe_fromGTF_no_chrY)

ri_fromGTF_no_chrY          <- fromGTF.RI[RI.keepAllJunctionsButChrY,]
ri_jc_ijc_no_chrY           <- ri.jc.ijc2[RI.keepAllJunctionsButChrY,]
ri_jc_sjc_no_chrY           <- ri.jc.sjc2[RI.keepAllJunctionsButChrY,]
rownames(ri_jc_ijc_no_chrY) <- rownames(ri_fromGTF_no_chrY)
rownames(ri_jc_sjc_no_chrY) <- rownames(ri_fromGTF_no_chrY)

se_fromGTF_no_chrY          <- fromGTF.SE[SE.keepAllJunctionsButChrY,]
se_jc_ijc_no_chrY           <- se.jc.ijc2[SE.keepAllJunctionsButChrY,]
se_jc_sjc_no_chrY           <- se.jc.sjc2[SE.keepAllJunctionsButChrY,]
rownames(se_jc_ijc_no_chrY) <- rownames(se_fromGTF_no_chrY)
rownames(se_jc_sjc_no_chrY) <- rownames(se_fromGTF_no_chrY)

dim(a3ss_fromGTF_no_chrY)
dim(a5ss_fromGTF_no_chrY)
dim(mxe_fromGTF_no_chrY)
dim(ri_fromGTF_no_chrY)
dim(se_fromGTF_no_chrY)

dim(a3ss_jc_ijc_no_chrY)
dim(a3ss_jc_sjc_no_chrY)

dim(a5ss_jc_ijc_no_chrY)
dim(a5ss_jc_sjc_no_chrY)

dim(mxe_jc_ijc_no_chrY)
dim(mxe_jc_sjc_no_chrY)

dim(ri_jc_ijc_no_chrY)
dim(ri_jc_sjc_no_chrY)

dim(se_jc_ijc_no_chrY)
dim(se_jc_sjc_no_chrY)

[1] 8920   11

[1] 5584   11

[1] 2979   13

[1] 6312   11

[1] 42611    11

...

[1] 2971 7020

[1] 6297 7020

[1] 6297 7020

[1] 42484  7020

[1] 42484  7020

## Exploratory and Differential analysis as_event:ijc, sjc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the included exon junction count expression; sex denotes the reported sex of the subject

## Differential analysis as_event (combined ijc and sjc)

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + B2 as_event + B3 sex*as_event + epsilon (error)
           

where y is the alternative splicing event expression; sex denotes the reported sex of the subject, as_event represents the specific alternative splicing event - either included exon junction counts or skipped exon junction counts and their interaction terms.   Donor is added to our model as a blocking variable used in both the calculation of duplicate correlation as well as in the linear fit.

### Voom, limma's lmFit and eBayes

Using sample as a blocking variable, we are able to model the effects of the donor on the results, which improves the power.  This topic is discussed in biostars https://www.biostars.org/p/54565/.  And Gordon Smyth answers the question here https://mailman.stat.ethz.ch/pipermail/bioconductor/2014-February/057887.html.  The method of modeling is a random effects approach in which the intra-donor correlation is incorporated into the covariance matrix instead of the linear predictor.   And though as Gordon Smyth states both are good method and the twoway anova approach makes fewer assumptions, the random effects approach is statistically more powerful.  

We have a balanced design in which all donors receive all stimuli (which is really in healthy human donors, life and all of its factors!) Our measurement has so many points -- we are measuring in the skipped exon approach, 42,611 junctions!   It is not possible to encorporate those measurements into the linear predictor.  A two-way ANOVA approach is virtually as powerful as the random effects approach 
and hence is preferable as it makes fewer assumptions.

For an unbalanced design in which each donor receives only a subset of the stimula, the random effects approach is more powerful.

Random effects approach is equivalent to The first method is twoway anova, a generalization of a paired analysis.


In [54]:
print_exploratory_plots <- function (plot, dup, tissue_of_interest, splice_type, fromGTF, tissue_list, ijc, sjc, obj, metadata ) {

    fromGTF           <- fromGTF
    tissue_true       <- tissue_list == tissue_of_interest

    # test to make sure we don't have nonsense
    # like proteins - this process moves in one direction
    # tissue will get us the phenotype selection 
    # from the phenotype SAMPID we get the metadata SAMPID
    # which leads us to counts - it goes in this direction.
    # first phenotype obj subsetting for the tissue_of_interest

    message("\nLimiting phenotype data to tissue of interest\n",
           paste(tissue_of_interest),collapse=" ")    
    keep = pData(obj)$SMTSD== tissue_of_interest
    message("\nkeep\n",
           paste(table(keep)),collapse=" ")    
    tissue_obj<- obj[,keep]
    message("\ntissue_obj now reduced to the tissue of interest\n",
            paste(dim(pData(tissue_obj)), collapse=" "))
    message("\nsample value\n",
            paste(   pData(tissue_obj)[1,], collapse=" "))

    # now we get the runs via the metadata matrix via SAMPID
    # limit the phenotype info to just the samples we care about
    metadata_samples          <- as.character(metadata$SAMPID)
    tissue_phenotype_samples  <- as.character(pData(tissue_obj)$SAMPID)

    rm(keep)
    keep = tissue_phenotype_samples %in% metadata_samples
    message("\nLimit the phenotypes to those we have samples for\n", 
        paste(table(keep), collapse = " ") )
    message("\ntissue_count_obj now has these count specific phenotypes")    
    tissue_count_obj        <- tissue_obj[keep,]
    pData(tissue_count_obj) <- pData(tissue_obj)[keep,]
    message("\nDimensions of tissue_count_obj\n", 
        paste(dim(pData(tissue_count_obj)), collapse = " ") )
    message("\nDimensions of tissue_count_obj \n", 
        paste( dim(tissue_count_obj), collapse = " ") )

    # and vice-versa, limit our samples to the ones we have phenotype for
    rm(keep)
    metadata_samples       <- as.character(metadata$SAMPID)
    phenotype_samples      <- as.character(pData(tissue_count_obj)$SAMPID)
    keep = metadata_samples %in% phenotype_samples
    message("\nLimit the counts now has these count specific phenotypes \n", 
        paste( table(keep), collapse = " ") )
    table(keep)

    ijc_tissue        <- ijc     [          ,keep==TRUE]
    sjc_tissue        <- sjc     [          ,keep==TRUE]
    metadata_tissue   <- metadata[keep==TRUE,          ]

    message("\ndimensions of the ijc_tissue \n", 
        paste(  dim(ijc_tissue), collapse = " ") )
    message("\ndimensions of the ijc_tissue \n", 
        paste(  dim(sjc_tissue), collapse = " ") )
    message("\ndimensions of the metadata_tissue \n", 
        paste(  dim(metadata_tissue), collapse = " ") )
   
    metadata_samples       <- as.character(metadata_tissue$SAMPID)
    phenotype_samples      <- as.character(pData(tissue_count_obj)$SAMPID)

    length(metadata_samples)
    length(phenotype_samples)
    non_overlaps <- setdiff( metadata_samples, phenotype_samples)
    message("\nThe non-overlapping IDs between pheno and count data are:\n", 
        paste(length(non_overlaps), collapse = " ") )

    ijc.dm            <- data.matrix(ijc_tissue)
    sjc.dm            <- data.matrix(sjc_tissue)    

    for (i in (1:length(metadata_samples))) {
        sample = metadata_tissue[i,]$SAMPID
        sample_sex = pData(tissue_count_obj)$SEX[pData(tissue_count_obj)$SAMPID == sample]
        if (i==1) {
            sex = sample_sex
        } else {
            sex = c(sex,sample_sex)
        }
    }
    message("sex samples:\n",
        paste0(table(sex), collapse="\n"))
    sex      <- ifelse(sex == 1,"male","female")
    sex      <- factor(sex,levels=c("male","female"))
    message("\nsex samples:\n",
        paste0(table(sex), collapse=" "))

    design    <- model.matrix ( ~ sex)
    message("\ndesign matrix ijc, alone:\n",
        paste0(head(design), collapse="\n"))

    colnames(design) <- c("intercept","sex")

    y_ijc <- DGEList(counts=ijc.dm, group = sex)
    y_ijc <- calcNormFactors(y_ijc, method="RLE")
    y_ijc_voom <- voom (y_ijc, design=design, plot=plot)

    Gender <- substring(sex,1,1)
    if (dup == TRUE) {
        pdf_sub_directory = '../pdf_dup_corr/'
        csv_sub_directory = '../csv_dup_corr/'
        message("mkdir -p pdf_sub_directory")
        system ("mkdir -p '../pdf_dup_corr/'")
        message("mkdir -p csv_sub_directory")
        system ("mkdir -p '../csv_dup_corr/'")
    } else {
        pdf_sub_directory = '../pdf_no_dup_corr/'
        csv_sub_directory = '../csv_no_dup_corr/'
        message("mkdir -p pdf_sub_directory")
        system ("mkdir -p '../pdf_no_dup_corr/'")
        message("mkdir -p csv_sub_directory")
        system ("mkdir -p '../csv_no_dup_corr/'")
    }
    
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                                     snakecase::to_snake_case(tissue_of_interest)),"-ijc-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y_ijc, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                                     snakecase::to_snake_case(tissue_of_interest)),"-ijc-voom-MDSplot-100.pdf")
    pdf (filename)    
        plotMDS(y_ijc_voom, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()
 
    fit_ijc <- lmFit(y_ijc_voom, design)
    fit_ijc <- eBayes(fit_ijc)

    ijc_sex_results                    <- topTable(fit_ijc, coef='sex', number=nrow(y_ijc_voom))
    ijc_sex_results_refined            <- ijc_sex_results$adj.P.Val <= 0.05 & abs(ijc_sex_results$logFC) >= abs(log2(1.5))
    ijc_sex_rnResults                  <- rownames(ijc_sex_results)
    ijc_sex_resultsAnnotations         <- fromGTF[ijc_sex_rnResults,]

    ijc_sex_results_refinedAnnotations <- ijc_sex_resultsAnnotations[ijc_sex_results_refined      ==TRUE,]
    message("\ndimensions of the ijc_sex_results_refined_annotations \n", 
        paste(dim (ijc_sex_results_refinedAnnotations), collapse = " ") )

    # geneSymbols are in the annotations 
    ijc_sex_geneSymbols               <- ijc_sex_resultsAnnotations$geneSymbol
    ijc_sex_refined_geneSymbols       <- ijc_sex_results_refinedAnnotations$geneSymbol
    message("\nlength ijc_sex_results_refined_geneSymbols\n", 
        paste(length(ijc_sex_refined_geneSymbols), collapse = " ") )

    # adjust the rownames to be the geneSymbols rather than junction IDs
    ijc_sex_results_rn         <- paste(ijc_sex_geneSymbols,       ijc_sex_rnResults, sep="-")
    rownames(ijc_sex_results)  <- ijc_sex_results_rn    

    ijc_sex_filename               = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_ijc_sex.csv',sep='')
    ijc_sex_refined_filename       = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_ijc_sex_refined.csv',sep='')
    ijc_sex_genesFilename          = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_ijc_sex_universe.txt',sep='')
    ijc_sex_refined_genesFilename  = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_ijc_sex_gene_set.txt',sep='')
    write.table(ijc_sex_results, 
                file = ijc_sex_filename        , row.names = T, col.names = T, quote = F, sep = ",")
    write.table(ijc_sex_results [ijc_sex_results_refined      ,], 
                file = ijc_sex_refined_filename, row.names = T, col.names = T, quote = F, sep = ",")
    write.table(ijc_sex_geneSymbols,        
                file = ijc_sex_genesFilename        , row.names = F, col.names = F, quote = F, sep = ",")
    write.table(ijc_sex_refined_geneSymbols,
                file = ijc_sex_refined_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
   
    message("\nstarting sjc\n")

    y_sjc <- DGEList(counts=sjc.dm, group = sex)
    y_sjc <- calcNormFactors(y_sjc, method="RLE")
    y_sjc_voom <- voom (y_sjc, design=design, plot=plot)

    Gender <- substring(sex,1,1)
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                                     snakecase::to_snake_case(tissue_of_interest)),"-sjc-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y_sjc, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                                     snakecase::to_snake_case(tissue_of_interest)),"-sjc-voom-MDSplot-100.pdf")
    pdf (filename)    
        plotMDS(y_sjc_voom, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()
 
    fit_sjc <- lmFit(y_sjc_voom, design)
    fit_sjc <- eBayes(fit_sjc)

    sjc_sex_results                    <- topTable(fit_sjc, coef='sex', number=nrow(y_sjc_voom))
    sjc_sex_results_refined            <- sjc_sex_results$adj.P.Val <= 0.05 & abs(sjc_sex_results$logFC) >= abs(log2(1.5))
    sjc_sex_rnResults                  <- rownames(sjc_sex_results)
    sjc_sex_resultsAnnotations         <- fromGTF[sjc_sex_rnResults,]

    sjc_sex_results_refinedAnnotations <- sjc_sex_resultsAnnotations[sjc_sex_results_refined      ==TRUE,]
    message("\ndimensions of the sjc_sex_results_refined_annotations \n", 
        paste(dim (sjc_sex_results_refinedAnnotations), collapse = " ") )

    # geneSymbols are in the annotations 
    sjc_sex_geneSymbols               <- sjc_sex_resultsAnnotations$geneSymbol
    sjc_sex_refined_geneSymbols       <- sjc_sex_results_refinedAnnotations$geneSymbol
    message("\nlength sjc_sex_results_refined_geneSymbols\n", 
        paste(length(sjc_sex_refined_geneSymbols), collapse = " ") )

    # adjust the rownames to be the geneSymbols rather than junction IDs
    sjc_sex_results_rn         <- paste(sjc_sex_geneSymbols, sjc_sex_rnResults, sep="-")
    rownames(sjc_sex_results)  <- sjc_sex_results_rn    

    sjc_sex_filename               = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sjc_sex.csv',sep='')
    sjc_sex_refined_filename       = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sjc_sex_refined.csv',sep='')
    sjc_sex_genesFilename          = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sjc_sex_universe.txt',sep='')
    sjc_sex_refined_genesFilename  = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sjc_sex_gene_set.txt',sep='')
    write.table(sjc_sex_results, 
                file = sjc_sex_filename        , row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sjc_sex_results [sjc_sex_results_refined      ,], 
                file = sjc_sex_refined_filename, row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sjc_sex_geneSymbols,        
                file = sjc_sex_genesFilename        , row.names = F, col.names = F, quote = F, sep = ",")
    write.table(sjc_sex_refined_geneSymbols,
                file = sjc_sex_refined_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
    
   
    message("starting the y prediction\n")
    sample_names <- as.character(colnames(ijc.dm))
    # we will add donor as a blocking parameter
    # rather than sample name -- we should use donor for real
    sample     <- factor(sample_names)
    
    donor    <- rep(sample, 2)
    message("\ndonor size", 
        paste(length(donor), collapse = " ") )
    
    ijc_names <- as.character(colnames(ijc.dm))
    sjc_names <- as.character(colnames(sjc.dm))
    sjc_names <- paste0(sjc_names,"-sjc")
    ijc_names <- paste0(ijc_names,"-ijc")

    colnames(ijc.dm) <- ijc_names
    colnames(sjc.dm) <- sjc_names

    as_matrix <- cbind(ijc.dm,sjc.dm)
    message("\ndim as_matrix", 
        paste(dim(as_matrix), collapse = " ")) 
            
    for (i in (1:length(metadata_samples))) {
        sample = metadata_tissue[i,]$SAMPID
        sample_sex = pData(tissue_count_obj)$SEX[pData(tissue_count_obj)$SAMPID == sample]
        if (i==1) {
            sex = sample_sex
        } else {
            sex = c(sex,sample_sex)
        }
    }
    message("sex samples:\n",
        paste0(table(sex), collapse="\n"))
    sex      <- ifelse(sex == 1,"male","female")
    sex      <- factor(sex,levels=c("male","female"))
    message("\nsex samples:\n",
        paste0(table(sex), collapse=" "))
    sex2      <- c(rep(sex,2))
    table(sex2)
    message("\nlength sex2\n", 
        paste(length(sex2), collapse = " ") )
    message("table sex2\n", 
        paste(table(sex2), collapse = "\n") )
    as_event  <- c(rep("ijc",dim(ijc.dm)[2]), rep("sjc", dim(sjc.dm)[2]))
    as_event  <- factor(as_event, levels=c("ijc", "sjc"))
    message("\nlength as_event\n", 
        paste(length(as_event), collapse = " ") )

    design    <- model.matrix( ~ sex2 + as_event + sex2*as_event)
    dim(design)
    colnames(design) <- c("intercept","sex", "as_event","sex*as_event")
    message("\ndim design <- model.matrix( ~sex + as_event + sex*as_event)\n", 
        paste(head(design), collapse = "\n") )

    y <- DGEList(counts=as_matrix, group = sex2)
    y <- calcNormFactors(y, method="RLE")
    y_voom <- voom (y, design=design, plot = plot)

    if (dup==TRUE) {
        dup_cor <- duplicateCorrelation(y_voom$E, design=design, ndups=2, block=donor, weights=y$samples$norm.factors)
        dup_cor$consensus.correlation 
        y_dup_voom <- voom (y, design=design, plot = plot, block = donor, correlation = dup_cor$consensus.correlation) 
    }
    
    Gender <- substring(sex[1:dim(ijc.dm)[2]],1,1)
    message("\nGenders new size\n", 
        paste(length(Gender), collapse = " ") )
    message("\nplotting y for ijc portion of design <- model.matrix( ~sex + as_event + sex*as_event\n")
     # print the combined exploratory plot
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                              snakecase::to_snake_case(tissue_of_interest)),"-y-ijc-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y[,c(1:dim(ijc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
            gene.selection="common")
    dev.off()
    message("\nplotting y_voom for ijc portion of design <- model.matrix( ~sex + as_event + sex*as_event\n")
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                              snakecase::to_snake_case(tissue_of_interest)),"-y-voom-ijc-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y_voom[,c(1:dim(ijc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
            gene.selection="common")
    dev.off()
    if (dup == TRUE) {
        filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                           snakecase::to_snake_case(tissue_of_interest)),"-y-dup-voom-ijc-MDSplot-100.pdf")
        pdf (filename)
            plotMDS(y_dup_voom[,c(1:dim(ijc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
        dev.off()
    }
    message("\nplotting y for sjc portion of design <- model.matrix( ~sex + as_event + sex*as_event\n")
    filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                              snakecase::to_snake_case(tissue_of_interest)),"-y-sjc-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y[,c((dim(ijc.dm)[2]+1)):(dim(ijc.dm)[2]+dim(sjc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
            gene.selection="common")
    dev.off()
    
    if (dup == TRUE) {
        message("\nplotting y_voom for sjc portion of design <- model.matrix( ~sex + as_event + sex*as_event\n")    
        filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                              snakecase::to_snake_case(tissue_of_interest)),"-y-voom-sjc-MDSplot-100.pdf")
        pdf (filename)
            plotMDS(y_voom[,c((dim(ijc.dm)[2]+1)):(dim(ijc.dm)[2]+dim(sjc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
        dev.off()
        filename <- paste0(paste0(paste0(pdf_sub_directory, splice_type),
                              snakecase::to_snake_case(tissue_of_interest)),"-y-dup-voom-sjc-MDSplot-100.pdf")
        pdf (filename)
            plotMDS(y_dup_voom[,c((dim(ijc.dm)[2]+1)):(dim(ijc.dm)[2]+dim(sjc.dm)[2])], labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
        dev.off()
        
        fit <- lmFit(y_dup_voom, design=design, block=donor, correlation = dup_cor$consensus.correlation)
    } else {
        fit <- lmFit(y_voom, design=design)
    }
        
    fit <- eBayes(fit, robust=TRUE)
    
    sex_as_events_results         <- topTable(fit, coef="sex*as_event", number=nrow(y_voom))
    sex_as_events_results_refined <- sex_as_events_results$adj.P.Val <= 0.05 & abs(sex_as_events_results$logFC) >= abs(log2(1.5))

    sex_results                   <- topTable(fit, coef="sex", number=nrow(y_voom))
    sex_results_refined           <- sex_results$adj.P.Val <= 0.05 & abs(sex_results$logFC) >= abs(log2(1.5))

    sex_as_events_rnResults <- rownames(sex_as_events_results)
    sex_rnResults           <- rownames(sex_results)
    head(sex_as_events_rnResults)
    head(ijc_sex_rnResults)
    head(sex_rnResults)
    head(fromGTF[sex_as_events_rnResults,])

    # use the junctionIDs to get the annotations
    sex_as_events_resultsAnnotations      <- fromGTF[sex_as_events_rnResults,]
    sex_resultsAnnotations                <- fromGTF[sex_rnResults,]
    ijc_sex_resultsAnnotations            <- fromGTF[ijc_sex_rnResults,]
    head(sex_as_events_resultsAnnotations)
    head(sex_resultsAnnotations)
    head(ijc_sex_resultsAnnotations)
    
    sex_as_events_results_refinedAnnotations<- sex_as_events_resultsAnnotations[sex_as_events_results_refined==TRUE,]
    sex_results_refinedAnnotations          <- sex_resultsAnnotations          [sex_results_refined          ==TRUE,]
    sjc_sex_results_refinedAnnotations       <- sjc_sex_resultsAnnotations      [sjc_sex_results_refined      ==TRUE,]
    head(sex_as_events_results_refinedAnnotations)
    head(sex_results_refinedAnnotations)
    head(sjc_sex_results_refinedAnnotations)

    # geneSymbols are in the annotations 
    sex_as_events_geneSymbols         <- sex_as_events_resultsAnnotations$geneSymbol
    sex_as_events_refined_geneSymbols <- sex_as_events_results_refinedAnnotations$geneSymbol
    sex_geneSymbols                   <- sex_resultsAnnotations$geneSymbol
    sex_refined_geneSymbols           <- sex_results_refinedAnnotations$geneSymbol
    ijc_sex_geneSymbols               <- ijc_sex_resultsAnnotations$geneSymbol
    ijc_sex_refined_geneSymbols       <- ijc_sex_results_refinedAnnotations$geneSymbol
    sjc_sex_geneSymbols               <- sjc_sex_resultsAnnotations$geneSymbol
    sjc_sex_refined_geneSymbols       <- sjc_sex_results_refinedAnnotations$geneSymbol


    # adjust the rownames to be the geneSymbols rather than junction IDs
    sex_as_events_results_rn   <- paste(sex_as_events_geneSymbols, sex_as_events_rnResults, sep="-")
    sex_results_rn             <- paste(sex_geneSymbols,           sex_rnResults, sep="-")
    ijc_sex_results_rn         <- paste(ijc_sex_geneSymbols,       ijc_sex_rnResults, sep="-")
    sjc_sex_results_rn         <- paste(sjc_sex_geneSymbols,       sjc_sex_rnResults, sep="-")
    message("\n sex_as_events\n", 
        paste(head(sex_as_events_results_rn), collapse = " ") )
    message("\n ijc_sex_results\n", 
        paste(head(ijc_sex_results_rn), collapse = " ") )
    message("\n sjc_sex_results\n", 
        paste(head(sjc_sex_results_rn), collapse = " ") )
    rownames(sex_as_events_results) <- sex_as_events_results_rn
    rownames(sex_results)           <- sex_results_rn
    rownames(ijc_sex_results)       <- ijc_sex_results_rn
    rownames(sjc_sex_results)       <- sjc_sex_results_rn

    sex_as_events_filename         = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sex_as_events.csv')
    sex_as_events_refined_filename = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sex_as_events_refined.csv',sep='')
    sex_filename                   = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sex.csv',sep='')
    sex_refined_filename           = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_DGE_sex_refined.csv',sep='')
    sex_as_events_genesFilename    = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sex_as_events_universe.txt',sep='')
    sex_as_events_refined_genesFilename = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sex_as_events_gene_set.txt',sep='')
    sex_genesFilename              = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sex_universe.txt',sep='')
    sex_refined_genesFilename      = paste0(paste0(paste0(csv_sub_directory, splice_type),
                                                   snakecase::to_snake_case(tissue_of_interest)),'_sex_gene_set.txt',sep='')

    write.table(sex_as_events_results, file = sex_as_events_filename, 
                row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sex_as_events_results[sex_as_events_results_refined,], 
                file = sex_as_events_refined_filename, row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sex_results,           file = sex_filename          , 
                row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sex_results [sex_results_refined          ,], file = sex_refined_filename, 
                row.names = T, col.names = T, quote = F, sep = ",")
    write.table(sex_as_events_geneSymbols, file = sex_as_events_genesFilename, 
                row.names = F, col.names = F, quote = F, sep = ",")
    write.table(sex_as_events_refined_geneSymbols,file = sex_as_events_refined_genesFilename, 
                row.names = F, col.names = F, quote = F, sep = ",")
    write.table(sex_geneSymbols,           file = sex_genesFilename          , 
                row.names = F, col.names = F, quote = F, sep = ",")
    write.table(sex_refined_geneSymbols,          file = sex_refined_genesFilename          , 
                row.names = F, col.names = F, quote = F, sep = ",")

    return(0)
}

#### Testing Snippet

To use this snippet - set *test = `TRUE`*.
To play with using or not using duplicate correlation, set *dup = `TRUE`* accordingly.

In [ ]:
#     Testing
# set test = TRUE to test
# set test = FALSE to ignore

test = TRUE
dup  = TRUE
tissue_list  <- factor(snakecase::to_snake_case(as.character(tissue_reduction$SMTSD)))
tissue_list
length(tissue_list)

if (test == TRUE) {
    splice_list       = c("a3ss_","a5ss_","mxe_","ri_","se_")

    splice_index        = 5
    tissue_index        = 21
    plot                = TRUE
    splice_type         = splice_list    [splice_index] 
    tissue_of_interest  = tissue_list[tissue_index]
    fromGTF             = se_fromGTF_no_chrY
    metadata            = ocounts_metadata
    ijc                 = se_jc_ijc_no_chrY

    message("\nijc\n", 
        paste(dim(ijc), collapse = "\n") )
    message("\nhead(ijc)\n", 
        paste(ijc[1:5,1:5], collapse = " ") )
    sjc                 = se_jc_sjc_no_chrY
    message("\nsjc\n", 
        paste(dim(sjc), collapse = "\n") )
    message("\nsjc\n", 
        paste(sjc[1:5,1:5], collapse = " ") )
    tissue_list         = tissue_list
    obj                 = reduced_obj2
    message("\nobj\n", 
        paste(dim(obj), collapse = "\n") )

    
    print_exploratory_plots (plot,
                             dup,
                             tissue_of_interest, 
                             splice_type, 
                             fromGTF, 
                             tissue_list, 
                             ijc, 
                             sjc, 
                             obj, 
                             metadata )
    message("\ndone!\n") 
    
}

## Execution of All Tissues and All Splicing Variants

Additional values set to enable this notebook to be executed as a nextflow workflow or to run in place with appropriate settings.

### parameters Setting

1. Setting `dup=TRUE` causes lengthy execution times.

2. Setting `plot=TRUE` can overwhelm the saving capacity within a jupyter-lab notebook - 
   this sets to print all the voom plots.
   
3. Adjusting `splice_type` will allow you to play with a variety of results

   a. all splice types desired to be run:
    
    `splice_list       = c("a3ss_","a5ss_","mxe_","ri_","se_")`
    
   b. a subset (leaving out say `splice_type = "se_"` since it is the largest, for example)
    
    `splice_list       = c("a3ss_","a5ss_","mxe_","ri_")`
    
   

**output directories**

### analysis results

1. csv results will be either:

   a. `../csv_no_dup` *when `dup == FALSE`*
   b. `../csv_dup`    *when `dup == TRUE`*

2. pdf results will be either:tissue_index -- enables this to run as a NextFlow notebook
 
   a. `../pdf_no_dup` *when `dup == FALSE`*
   b. `../pdf_dup`    *when `dup == TRUE`*


In [56]:
tissue_list  <- factor(snakecase::to_snake_case(as.character(tissue_reduction$SMTSD)))
tissue_list
length(tissue_list)
pdata_tissue_list <- factor(snakecase::to_snake_case(as.character(pData(reduced_obj2)$SMTSD)))
levels(pdata_tissue_list)
length(levels(pdata_tissue_list))


[1] adipose_subcutaneous                  adipose_visceral_omentum             
 [3] adrenal_gland                         artery_aorta                         
 [5] artery_coronary                       artery_tibial                        
 [7] brain_caudate_basal_ganglia           brain_cerebellar_hemisphere          
 [9] brain_cerebellum                      brain_cortex                         
[11] brain_frontal_cortex_ba_9             brain_hippocampus                    
[13] brain_hypothalamus                    brain_nucleus_accumbens_basal_ganglia
[15] brain_putamen_basal_ganglia           brain_spinal_cord_cervical_c_1       
[17] breast_mammary_tissue                 cells_cultured_fibroblasts           
[19] cells_ebv_transformed_lymphocytes     colon_sigmoid                        
[21] colon_transverse                      esophagus_gastroesophageal_junction  
[23] esophagus_mucosa                      esophagus_muscularis                 
[25] heart_atrial_appendage                heart_left_ventricle                 
[27] liver                                 lung                                 
[29] muscle_skeletal                       nerve_tibial                         
[31] pancreas                              pituitary                            
[33] skin_not_sun_exposed_suprapubic       skin_sun_exposed_lower_leg           
[35] small_intestine_terminal_ileum        spleen                               
[37] stomach                               thyroid                              
[39] whole_blood                          
39 Levels: adipose_subcutaneous adipose_visceral_omentum ... whole_blood

[1] 39

[1] "adipose_subcutaneous"                 
 [2] "adipose_visceral_omentum"             
 [3] "adrenal_gland"                        
 [4] "artery_aorta"                         
 [5] "artery_coronary"                      
 [6] "artery_tibial"                        
 [7] "brain_caudate_basal_ganglia"          
 [8] "brain_cerebellar_hemisphere"          
 [9] "brain_cerebellum"                     
[10] "brain_cortex"                         
[11] "brain_frontal_cortex_ba_9"            
[12] "brain_hippocampus"                    
[13] "brain_hypothalamus"                   
[14] "brain_nucleus_accumbens_basal_ganglia"
[15] "brain_putamen_basal_ganglia"          
[16] "brain_spinal_cord_cervical_c_1"       
[17] "breast_mammary_tissue"                
[18] "cells_cultured_fibroblasts"           
[19] "cells_ebv_transformed_lymphocytes"    
[20] "colon_sigmoid"                        
[21] "colon_transverse"                     
[22] "esophagus_gastroesophageal_junction"  
[23] "esophagus_mucosa"                     
[24] "esophagus_muscularis"                 
[25] "heart_atrial_appendage"               
[26] "heart_left_ventricle"                 
[27] "liver"                                
[28] "lung"                                 
[29] "muscle_skeletal"                      
[30] "nerve_tibial"                         
[31] "pancreas"                             
[32] "pituitary"                            
[33] "skin_not_sun_exposed_suprapubic"      
[34] "skin_sun_exposed_lower_leg"           
[35] "small_intestine_terminal_ileum"       
[36] "spleen"                               
[37] "stomach"                              
[38] "thyroid"                              
[39] "whole_blood"

[1] 39

In [57]:
# splice_list <- c("a3ss_","a5ss_","mxe_","ri_","se_")

splice_list  <- c("a3ss_","a5ss_","mxe_","ri_", "se_")

# parameters that do not change with splice_type
tissue_list  <- factor(snakecase::to_snake_case(as.character(tissue_reduction$SMTSD)))
tissue_list
length(tissue_list)

metadata     <- ocounts_metadata
plot         <- FALSE
dup          <- FALSE
obj          <- reduced_obj2

# parameters that change with each splice type (3)
# fromGTF
# ijc
# sjc

for (tissue_index in 1:length(tissue_list)) {

    # a3ss
    splice_type = "a3ss_"
    res = splice_list %in% splice_type
    tissue_of_interest  = as.vector(as.character(tissue_list[tissue_index]))
    message ("processing ",
             paste(tissue_of_interest), " \n")
    
    if (sum(res) == 1) {
        message ("splice_list does contain\n",
             paste(splice_type), " continuing with processing\n")
        fromGTF             = a3ss_fromGTF_no_chrY
        ijc                 = a3ss_jc_ijc_no_chrY
        sjc                 = a3ss_jc_sjc_no_chrY
        print_exploratory_plots (plot, 
                             dup,
                             tissue_of_interest, 
                             splice_type, 
                             fromGTF, 
                             tissue_list, 
                             ijc, 
                             sjc, 
                             obj, 
                             metadata )
    }
    # a5ss
    splice_type = "a5ss_"
    res = splice_list %in% splice_type
    if (sum(res) == 1) {
        message ("splice_list does contain\n",
             paste(splice_type), " continuing with processing\n")
        fromGTF             = a5ss_fromGTF_no_chrY
        ijc                 = a5ss_jc_ijc_no_chrY
        sjc                 = a5ss_jc_sjc_no_chrY
        print_exploratory_plots (plot, 
                             dup,
                             tissue_of_interest, 
                             splice_type, 
                             fromGTF, 
                             tissue_list, 
                             ijc, 
                             sjc, 
                             obj, 
                             metadata )
    }
    # mxe
    splice_type = "mxe_"
    res = splice_list %in% splice_type
    if (sum(res) == 1) {
        message ("splice_list does contain\n",
             paste(splice_type), " continuing with processing\n")
        fromGTF             = mxe_fromGTF_no_chrY
        ijc                 = mxe_jc_ijc_no_chrY
        sjc                 = mxe_jc_sjc_no_chrY
        print_exploratory_plots (plot, 
                             dup,
                             tissue_of_interest, 
                             splice_type, 
                             fromGTF, 
                             tissue_list, 
                             ijc, 
                             sjc, 
                             obj, 
                             metadata )
    }
    # ri
    splice_type = "ri_"
    res = splice_list %in% splice_type
    if (sum(res) == 1) {
        message ("splice_list does contain\n",
             paste(splice_type), " continuing with processing\n")
        fromGTF             = ri_fromGTF_no_chrY
        ijc                 = ri_jc_ijc_no_chrY
        sjc                 = ri_jc_sjc_no_chrY
        print_exploratory_plots (plot, 
                             dup,
                             tissue_of_interest, 
                             splice_type, 
                             fromGTF, 
                             tissue_list, 
                             ijc, 
                             sjc, 
                             obj, 
                             metadata )
    }
    # se
    splice_type = "se_"
    res = splice_list %in% splice_type
    if (sum(res) == 1) {
        message ("splice_list does contain\n",
             paste(splice_type), " continuing with processing\n")
        fromGTF             = se_fromGTF_no_chrY
        ijc                 = se_jc_ijc_no_chrY
        sjc                 = se_jc_sjc_no_chrY
        print_exploratory_plots (plot, 
                              dup,
                              tissue_of_interest, 
                              splice_type, 
                              fromGTF, 
                              tissue_list, 
                              ijc, 
                              sjc, 
                              obj, 
                              metadata )
    }
}

[1] adipose_subcutaneous                  adipose_visceral_omentum             
 [3] adrenal_gland                         artery_aorta                         
 [5] artery_coronary                       artery_tibial                        
 [7] brain_caudate_basal_ganglia           brain_cerebellar_hemisphere          
 [9] brain_cerebellum                      brain_cortex                         
[11] brain_frontal_cortex_ba_9             brain_hippocampus                    
[13] brain_hypothalamus                    brain_nucleus_accumbens_basal_ganglia
[15] brain_putamen_basal_ganglia           brain_spinal_cord_cervical_c_1       
[17] breast_mammary_tissue                 cells_cultured_fibroblasts           
[19] cells_ebv_transformed_lymphocytes     colon_sigmoid                        
[21] colon_transverse                      esophagus_gastroesophageal_junction  
[23] esophagus_mucosa                      esophagus_muscularis                 
[25] heart_atrial_appendage                heart_left_ventricle                 
[27] liver                                 lung                                 
[29] muscle_skeletal                       nerve_tibial                         
[31] pancreas                              pituitary                            
[33] skin_not_sun_exposed_suprapubic       skin_sun_exposed_lower_leg           
[35] small_intestine_terminal_ileum        spleen                               
[37] stomach                               thyroid                              
[39] whole_blood                          
39 Levels: adipose_subcutaneous adipose_visceral_omentum ... whole_blood

[1] 39

splice_list does contain
a3ss_ continuing with processing

processing adipose_subcutaneous 


Limiting phenotype data to tissue of interest
adipose_subcutaneous 

keep
9945427 

tissue_obj now reduced to the tissue of interest
427 68

sample value
GTEX.1117F.0226.SM.5GZZ7 c(`GTEX-1117F-0226-SM-5GZZ7` = 1) c(`GTEX-1117F-0226-SM-5GZZ7` = 1) c(`GTEX-1117F-0226-SM-5GZZ7` = 1657) c(`GTEX-1117F-0226-SM-5GZZ7` = 23) c(`GTEX-1117F-0226-SM-5GZZ7` = 1) 1 c(`GTEX-1117F-0226-SM-5GZZ7` = 30) c(`GTEX-1117F-0226-SM-5GZZ7` = 1393) c(`GTEX-1117F-0226-SM-5GZZ7` = 812) c(`GTEX-1117F-0226-SM-5GZZ7` = 1045) c(`GTEX-1117F-0226-SM-5GZZ7` = 6) c(`GTEX-1117F-0226-SM-5GZZ7` = 595) c(`GTEX-1117F-0226-SM-5GZZ7` = 261) c(`GTEX-1117F-0226-SM-5GZZ7` = 79) c(`GTEX-1117F-0226-SM-5GZZ7` = 18) c(`GTEX-1117F-0226-SM-5GZZ7` = 3) c(`GTEX-1117F-0226-SM-5GZZ7` = NA) c(`GTEX-1117F-0226-SM-5GZZ7` = 4430) c(`GTEX-1117F-0226-SM-5GZZ7` = 10620) c(`GTEX-1117F-0226-SM-5GZZ7` = 5305) c(`GTEX-1117F-0226-SM-5GZZ7` = NA) c(`GTEX-1117F-

ERROR: Error in enc2utf8(string): argument is not a character vector


## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
rm (notebookid)
notebookid   = "AllTissueJunctionAnalysis"
notebookid

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

paste0("../metadata/", notebookid, "_sha256sums.txt")

data.table::fread(paste0("../metadata/", notebookid, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]